In [ ]:
import numpy as np

array = np.load('x_train.npy')

print(array[1])

In [13]:
import numpy as np
# Load the training data
x_train = np.load('x_train.npy').reshape(-1, 28*28)

labels = np.load('y_train.npy')  # Assuming you have labels for the data
# Concatenate the data of the three classes
finalLabels = []
class_indices = {str(i): [] for i in range(3)}  # Assuming you have only 3 classes
for i, label in enumerate(labels):
    if (label < 3):  # Considering only the first 3 classes
        class_indices[str(label)].append(i)

data_class_0 = x_train[class_indices['0']]
data_class_1 = x_train[class_indices['1']]
data_class_2 = x_train[class_indices['2']]

label_class_0 = labels[class_indices['0']]
label_class_1 = labels[class_indices['1']]
label_class_2 = labels[class_indices['2']]

labels = np.concatenate((label_class_0, label_class_1, label_class_2), axis=0)


# Concatenate data from all three classes
X = np.concatenate((data_class_0, data_class_1, data_class_2), axis=0)
print(X.shape)


(18623, 784)


In [14]:
def pca(data, n_components=10):
    # Step 1: Calculate mean of the data
    data_mean = np.mean(data, axis=0)
    
    # Step 2: Center the data
    data_centered = data - data_mean
    
    # Step 3: Compute covariance matrix
    covariance_matrix = np.cov(data_centered, rowvar=False)
    
    # Step 4: Compute eigenvectors and eigenvalues
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    
    # Step 5: Sort eigenvectors and eigenvalues in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]
    
    # Step 6: Choose top n_components eigenvectors
    U = sorted_eigenvectors[:, :n_components]
    
    # Step 7: Project the data onto the top n_components eigenvectors
    Y = np.dot(U.T, data_centered.T)
    
    # Step 8: Reconstruct the data
    x_reconstructed = np.dot(U, Y) + data_mean.reshape(-1, 1)
    
    # Step 9: Further reduce dimensionality if needed
    Up = U[:, :n_components]
    data_reduced = np.dot(Up.T, x_reconstructed)
    
    return Up.T , x_reconstructed
pca_matrix1, x_reconstructed = pca(X, n_components=10)
pca_matrix = np.dot(pca_matrix1, x_reconstructed).T
print(pca_matrix.shape)


(18623, 10)


In [5]:
# Load the test data
x_test = np.load('x_test.npy').reshape(-1, 28*28)
y_test = np.load('y_test.npy')

# Filter out samples from classes other than 0, 1, and 2
class_indices_test = {str(i): [] for i in range(3)}
for i, label in enumerate(y_test):
    if label < 3:  # Considering only the first 3 classes
        class_indices_test[str(label)].append(i)

# Extract data corresponding to classes 0, 1, and 2
data_class_0_test = x_test[class_indices_test['0']]
data_class_1_test = x_test[class_indices_test['1']]
data_class_2_test = x_test[class_indices_test['2']]

# Concatenate data from all three classes vertically
X_test = np.concatenate((data_class_0_test, data_class_1_test, data_class_2_test), axis=0)
y_test_filtered = y_test[np.concatenate((class_indices_test['0'], class_indices_test['1'], class_indices_test['2']))]

# Calculate accuracy



In [15]:
import numpy as np

def calculate_gini(labels):
    classes = np.unique(labels)
    gini = 0
    total_samples = len(labels)
    for c in classes:
        proportion = np.sum(labels == c) / total_samples
        gini += proportion * (1 - proportion)
    return gini

def find_best_split(X, y):
    best_gini = float('inf')
    best_feature = None
    best_threshold = None
    
    for feature in range(X.shape[1]):  # Loop over each feature
        thresholds = np.unique(X[:, feature])  # Unique values in the feature
        for threshold in thresholds:
            left_mask = X[:, feature] <= threshold
            right_mask = X[:, feature] > threshold
            
            left_gini = calculate_gini(y[left_mask])
            right_gini = calculate_gini(y[right_mask])
            
            total_gini = (len(y[left_mask]) * left_gini + len(y[right_mask]) * right_gini) / len(y)
            
            if total_gini < best_gini:
                best_gini = total_gini
                best_feature = feature
                best_threshold = threshold
                
    return best_feature, best_threshold

def grow_decision_tree(X, y, max_nodes=2):
    if max_nodes == 1 or len(np.unique(y)) == 1:
        # If reached maximum nodes or all labels are the same, return leaf node
        return {'class': np.argmax(np.bincount(y))}
    
    # Find the best split
    best_feature, best_threshold = find_best_split(X, y)
    
    if best_feature is None:
        # If no split improves purity, return leaf node
        return {'class': np.argmax(np.bincount(y))}
    
    # Split the data
    left_mask = X[:, best_feature] <= 90
    right_mask = X[:, best_feature] > 40
    
    # Recursively grow the left and right subtrees
    left_subtree = grow_decision_tree(X[left_mask], y[left_mask], max_nodes=max_nodes-1)
    right_subtree = grow_decision_tree(X[right_mask], y[right_mask], max_nodes=max_nodes-1)
    
    return {'feature': best_feature, 'threshold': best_threshold, 'left': left_subtree, 'right': right_subtree}

# Grow decision tree
decision_tree = grow_decision_tree(pca_matrix, labels, max_nodes=2)
print(decision_tree)


{'feature': 0, 'threshold': 33.76691272246312, 'left': {'class': 1}, 'right': {'class': 0}}


In [ ]:
# Load the test data
x_test = np.load('x_test.npy').reshape(-1, 28*28)
y_test = np.load('y_test.npy')

# Filter out samples from classes other than 0, 1, and 2
class_indices_test = {str(i): [] for i in range(3)}
for i, label in enumerate(y_test):
    if label < 3:  # Considering only the first 3 classes
        class_indices_test[str(label)].append(i)

# Extract data corresponding to classes 0, 1, and 2
data_class_0_test = x_test[class_indices_test['0']]
data_class_1_test = x_test[class_indices_test['1']]
data_class_2_test = x_test[class_indices_test['2']]

# Concatenate data from all three classes vertically
X_test = np.concatenate((data_class_0_test, data_class_1_test, data_class_2_test), axis=0)
y_test_filtered = y_test[np.concatenate((class_indices_test['0'], class_indices_test['1'], class_indices_test['2']))]

#calculate the pca_matrix for the test data
pca_matrix2, x_reconstructed2 = pca(X_test, n_components=10)
pca_matrix_test = np.dot(pca_matrix2, x_reconstructed2).T
# Calculate accuracy
print(pca_matrix_test.shape)
# Calculate accuracy for each class
class_accuracies = {}
for c in range(3):
    class_mask = (y_test_filtered == c)
    class_accuracy = calculate_accuracy(decision_tree, X_test[class_mask], y_test_filtered[class_mask])
    class_accuracies[c] = class_accuracy

print("Class-wise accuracies:", class_accuracies)

In [ ]:
print(X_test.shape)
print(y_test_filtered.shape)

(3147, 784)
(3147,)


In [ ]:
from sklearn.utils import resample
from collections import Counter

# Define function to perform majority voting
def majority_voting(predictions):
    # Perform majority voting
    voted_predictions = []
    for i in range(len(predictions[0])):
        votes = Counter([prediction[i] for prediction in predictions])
        majority_vote = votes.most_common(1)[0][0]
        voted_predictions.append(majority_vote)
    return voted_predictions

# Perform bagging with decision trees
num_trees = 5
tree_predictions = []

for _ in range(num_trees):
    # Generate a bootstrapped dataset
    bootstrapped_X, bootstrapped_y = resample(x_train, labels, replace=True)
    
    # Grow decision tree on bootstrapped dataset
    decision_tree = grow_decision_tree(bootstrapped_X, bootstrapped_y, max_nodes=3)
    
    # Predict class labels for test samples using decision tree
    predictions = []
    for sample in X_test:
        prediction = predict_labels(decision_tree, sample)
        predictions.append(prediction)
    
    # Append predictions to list
    tree_predictions.append(predictions)

# Perform majority voting
voted_predictions = majority_voting(tree_predictions)

# Calculate total accuracy
total_correct = sum(1 for i in range(len(y_test_filtered)) if voted_predictions[i] == y_test_filtered[i])
total_accuracy = total_correct / len(y_test_filtered)
print("Total accuracy:", total_accuracy)

# Calculate class-wise accuracy
class_correct = {c: 0 for c in range(3)}
class_total = {c: 0 for c in range(3)}
for i in range(len(y_test_filtered)):
    if voted_predictions[i] == y_test_filtered[i]:
        class_correct[y_test_filtered[i]] += 1
    class_total[y_test_filtered[i]] += 1

class_accuracy = {c: class_correct[c] / class_total[c] for c in range(3)}
print("Class-wise accuracies:", class_accuracy)
